In [1]:
FILE='../src/datas/BDALTIV/BDALTIV2_2-0_250M_ASC_LAMB93-IGN69_FRANCE_2018-01-15/BDALTIV2/1_DONNEES_LIVRAISON_2018-01-00246/BDALTIV2_MNT_250M_ASC_LAMB93_IGN69_FRANCE/BDALTIV2_250M_FXX_0098_7150_MNT_LAMB93_IGN69.asc'

In [2]:
import numpy as np
import pyproj
import pyvista as pv
import meshio

with open(FILE, "r") as fichier:
    # Lire les six premières lignes pour extraire les informations de la grille
    ncols = int(fichier.readline().split()[1])
    nrows = int(fichier.readline().split()[1])
    xllcorner = float(fichier.readline().split()[1])
    yllcorner = float(fichier.readline().split()[1])
    # Création du transformer de coordonnées
    in_proj = pyproj.Proj(init='epsg:2154')  # Lambert 93
    out_proj = pyproj.Proj(init='epsg:4326')  # WGS84
    lon, lat = pyproj.transform(in_proj, out_proj, xllcorner, yllcorner)
    cellsize = float(fichier.readline().split()[1])
    nodata_value = float(fichier.readline().split()[1])

    # Lire les données de la grille dans un tableau NumPy
    grille = np.zeros((nrows, ncols))
    for i in range(nrows):
        ligne = np.array([float(x) if float(x) != nodata_value else 0.0 for x in fichier.readline().split()])
        grille[i, :] = ligne

# Créer un objet UniformGrid à partir des données de la grille
grid = pv.UniformGrid()
grid.dimensions = (ncols, nrows, 1)
grid.origin = (lon, lat, 0) # longitude, latitude, altitude
grid.spacing = (cellsize, cellsize, 1)
grid.point_data["values"] = grille.flatten(order="F")


In [3]:
grid

UniformGrid (0x7fc163e17d60)
  N Cells:      21150801
  N Points:     21160000
  X Bounds:     -4.124e+00, 1.150e+06
  Y Bounds:     4.086e+01, 1.150e+06
  Z Bounds:     0.000e+00, 0.000e+00
  Dimensions:   4600, 4600, 1
  Spacing:      2.500e+02, 2.500e+02, 1.000e+00
  N Arrays:     1

In [4]:
grid.dimensions

(4600, 4600, 1)

In [5]:
subset = grid.extract_subset((0, 5000, 0, 5000, 0, 0), (5, 5, 1))
subset.plot(cpos="xy")


Widget(value="<iframe src='http://localhost:65509/index.html?ui=P_0x7fc161c67670_0&reconnect=auto' style='widt…

In [9]:
terrain = subset.warp_by_scalar()

In [10]:
terrain

StructuredGrid (0x7fc1386ba940)
  N Cells:      844561
  N Points:     846400
  X Bounds:     -4.124e+00, 1.149e+06
  Y Bounds:     4.086e+01, 1.149e+06
  Z Bounds:     -4.166e+01, 4.370e+03
  Dimensions:   920, 920, 1
  N Arrays:     1

In [11]:
terrain.plot()

Widget(value="<iframe src='http://localhost:65509/index.html?ui=P_0x7fc13120deb0_2&reconnect=auto' style='widt…

In [ ]:
# extraire la géométrie comme une PolyData
polydata = grid.extract_geometry()

# sauvegarder la PolyData au format STL
polydata.save('terrain.stl')